#### Side Input

In [2]:
import apache_beam as beam

side_list=list()
with open ('exclude_ids.txt','r') as my_file:
  for line in my_file:
    side_list.append(line.rstrip())

p = beam.Pipeline()

# We can pass side inputs to a ParDo transform, which will get passed to its process method.
# The first two arguments for the process method would be self and element.

class FilterUsingLength(beam.DoFn):
  def process(self, element,side_list,lower_bound, upper_bound=float('inf')):
    id = element.split(',')[0]
    name = element.split(',')[1]
    id=id.decode('utf-8','ignore').encode("utf-8")
    element_list= element.split(',')
    if (lower_bound <= len(name) <= upper_bound) and id not in side_list:
      return [element_list]

# using pardo to filter names with length between 3 and 10
small_names =( 
                p
                | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                | "ParDo with side inputs" >> beam.ParDo(FilterUsingLength(),side_list,3,10) 
                | beam.Filter(lambda record: record[3] == 'Accounts')
                | beam.Map(lambda record: (record[0]+ " " + record[1], 1))
                | beam.CombinePerKey(sum)
                | 'Write results' >> beam.io.WriteToText('data/output_new_final')
             )

p.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

ERROR:apache_beam.runners.common:'str' object has no attribute 'decode' [while running '[2]: ParDo with side inputs']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1435, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 851, in apache_beam.runners.common.PerWindowInvoker.invoke_process
  File "apache_beam/runners/common.py", line 997, in apache_beam.runners.common.PerWindowInvoker._invoke_process_per_window
  File "/tmp/ipykernel_14784/3511243644.py", line 17, in process
    id=id.decode('utf-8','ignore').encode("utf-8")
AttributeError: 'str' object has no attribute 'decode'. Did you mean: 'encode'?


AttributeError: 'str' object has no attribute 'decode' [while running '[2]: ParDo with side inputs']

#### Aditional Output

In [ ]:
import apache_beam as beam

# DoFn function 
class ProcessWords(beam.DoFn):
  
  def process(self, element, cutoff_length, marker):
    
    name = element.split(',')[1]
    
    if len(name) <= cutoff_length:
      return [beam.pvalue.TaggedOutput('Short_Names', name)]
    
    else:
      return [beam.pvalue.TaggedOutput('Long_Names', name)]
    
    if name.startswith(marker):
      return name 
      

p = beam.Pipeline()

      
results = (
            p
            | beam.io.ReadFromText('dept_data.txt')
    
            | beam.ParDo(ProcessWords(), cutoff_length=4, marker='A').with_outputs('Short_Names', 'Long_Names', main='Names_A')

          )

short_collection = results.Short_Names
long_collection = results.Long_Names
startA_collection = results.Names_A  

# write to file  
short_collection | 'Write 1'>> beam.io.WriteToText('short')

# write to file
long_collection | 'Write 2'>> beam.io.WriteToText('long')

# write to file
startA_collection | 'Write 3'>> beam.io.WriteToText('start_a')

p.run()

!{'head -n 5 short-00000-of-00001'}
!{'head -n 5 long-00000-of-00001'}
!{'head -n 5 start_a-00000-of-00001'}
